# Practical Session: Q-Learning

In this practical session, you will implement the famous [Q-Learning](https://link.springer.com/content/pdf/10.1007/BF00992698.pdf) algorithm and test it in various environments.

## Taxi V3

The taxi problem was first introduced in [Hierarchical Reinforcement Learning with the MAXQ Value Function Decomposition](https://arxiv.org/abs/cs/9905014).  
In this environment, the agent controls a taxi whose job is to pick up a passenger at one location and drop him off in his target location.  
Dropping of the passenger to its destination leads to a +20 reward.  
To encourage the agent to be as fast as possible, it receives a deceptive reward of -1 at each step.  
It will also receive a -10 penalty reward if it takes an illegal 'pick-up' or drop-off' action.  
Let's instantiate a Taxi-V3 environment:

In [ ]:
import gym
env = gym.make("Taxi-v3")
env.reset()
env.render()

The taxi is represented in <span style="color:yellow">yellow</span> when empty and in <span style="color:green">green</span> when full.  
The passenger is represented in <span style="color:blue">blue</span> and his destination in <span style="color:magenta">magenta</span>.

**Observations**: There are 500 discrete states since there are 25 taxi positions, five possible passenger locations (including the case when the passenger is in the taxi), and four destination locations.

In [ ]:
print(f"State Space {env.observation_space}")

The agent controling the *taxi* has six discrete and deterministic possibles actions:  
*  0: *move south*
*  1: *move north*
*  2: *move east*
*  3: *move west*
*  4: *pickup passenger*
*  5: *drop off passenger*  

In [ ]:
print(f"Action Space {env.action_space}")

Here is a little method to vizualize our taxi's trajectory:

In [ ]:
from IPython.display import clear_output
from time import sleep

def display_trajectory(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Reward: {frame['reward']}")
        sleep(.2)

# Random agent

The following code shows how to run an episode with an agent taking random actions.

In [ ]:
frames = [] # for animation

env.reset()
while True:
    # draw a random action from the action space
    action = env.action_space.sample()
    # the step method takes an action as input and returns 4 variables described in the OpenAI section
    state, reward, done, info = env.step(action)
    frames.append({
        'frame': env.render(mode='ansi'),
        'state': state,
        'reward': reward
        })
    #if done is True then the episode is over
    if done == True:
        break
        
display_trajectory(frames)

## Human policy 

In [ ]:
env.reset()
env.render()

We just saw a random policy. Let's now check on your policy.  
Using the possible actions, make a little script to take the client to his destination and visualize it with the `display_frames` method.

In [ ]:
frames = [] 
actions = [...]
for a in actions:
    ...

display_trajectory(frames)

# Q-learning
Let's now try to solve the TaxiV3 problem using the Q-learing algorithm.
![](images/q-learning.png)

Fill the `q_learning` method.

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import seaborn as sns

def print_running_mean(training_rewards, i):
    clear_output(wait=True)
    plt.figure(figsize=(15,3))
    plt.plot(pd.Series(training_rewards[:i]).rolling(100).mean())
    plt.title("Rewards running mean on last 100 episodes")
    plt.show()

    
def q_learning(env, alpha, gamma, epsilon, nb_episodes):
    nb_states = env.observation_space.n
    nb_actions = env.action_space.n
    q_table = np.zeros([nb_states, nb_actions])
    training_rewards = np.zeros(nb_episodes)
    for i in range(nb_episodes):
        ...
        while True:
            ...

            training_rewards[i] += reward
            if done:
                break

        if i % 100 == 0:
            print_running_mean(training_rewards, i)

    return q_table

Train your q-table.

In [ ]:
nb_episodes = 5000
alpha = 0.1
gamma = 0.99
epsilon = 0.1

q_table = ...

Let us have a look at the learned policy.

In [ ]:
env.reset()
frames = [] # for animation
timesteps = 0
while True:
    ...
        
display_trajectory(frames)

# Frozen lake
Now, try your algoithm on frozen lake.

In [ ]:
import gym
env = gym.make("FrozenLake8x8-v0")
env.render()

In [ ]:
nb_episodes = 20000
alpha = 0.1
gamma = 0.99
epsilon = 0.1

q_table = ...

Your agent has probably not reached a satisfying policy.  
This is due to the lack of exploration induced by the $\epsilon$-greedy policy.
Fixing $\epsilon$ to 0.1 is not sufficient in Frozen Lake. The agent does not explore enougth and thus never receives a positive feedback.  
A simple solution would be to use a decaying epsilon from 1 to 0.1.

In [ ]:
decay_rates = [0.9, 0.99, 0.9995, 0.9999]
eps_min = 0.1
plt.figure(figsize=(10,6))

for decay_rate in decay_rates:
    eps_list = []
    test_eps = 1
    for _ in range(nb_episodes):
        test_eps = max(test_eps * decay_rate, eps_min)
        eps_list.append(test_eps)          
    
    plt.plot(eps_list, label='decay rate: {}'.format(decay_rate))

plt.title('Effect of various decay rates')
plt.legend(loc='best')
plt.xlabel('# of episodes')
plt.ylabel('epsilon')
plt.show()

Modify your `q_learning` function to decay the learning rate after each episode (don't forget to clip so it does not go below 0.1).
Try your function with different values for `epsilon_decay`

In [ ]:
def q_learning(env, alpha, gamma, epsilon_decay, nb_episodes):
    epsilon = 1
    min_epsilon = 0.1
    nb_states = env.observation_space.n
    nb_actions = env.action_space.n
    q_table = np.zeros([nb_states, nb_actions])
    training_rewards = np.zeros(nb_episodes)
    for i in range(nb_episodes):
        ...
        ...
        ...
        if i % 100 == 0:
            print_running_mean(training_rewards, i)
            
    return q_table

In [ ]:
nb_episodes = 20000
alpha = 0.1
gamma = 0.99
epsilon_decay = 0.9999

q_table = q_learning(env, alpha, gamma, epsilon_decay, nb_episodes)

Let us have a look at the learned policy.

In [ ]:
env.reset()
frames = [] # for animation
timesteps = 0
state = env.reset()
while True:
    ...
        
display_trajectory(frames)

In these two environments, the State-space was discrete and small enough to learn the entire Q-table.
In most cases, this State space is way too large to learn the Q-values for each state.  
One solution is to learn an estimator of this Q-function.
In the following practical session, you will adapt the Q-learning algorithm to uses neural networks as function approximators.